# Statistics

### BUSI 520 - Python for Business Research
### Kerry Back, JGSB, Rice University

* $t$ tests with scipy.stats
* regressions with statsmodels formula API
    * HAC std errors
  * logit
* regressions with linearmodels
  * two-stage least squares
  * seemingly unrelated regressions
  * fixed effects and clustered std errors
  * Fama-MacBeth
* VARs with statsmodels
* saving output
  * to Excel
  * to latex with pystout

In [1]:
import numpy as np
import pandas as pd
import pandas_datareader as pdr

from scipy.stats import ttest_1samp, ttest_ind 
import statsmodels.api as sm
import statsmodels.formula.api as smf
from linearmodels.panel import PanelOLS, FamaMacBeth
from linearmodels.iv import IV2SLS
from linearmodels.system import SUR
from statsmodels.regression.rolling import RollingOLS
from statsmodels.tsa.api import VAR
from pystout import pystout
import openpyxl

## $t$ tests with scipy.stats

In [2]:
# use result.statistic and result.pvalue to get t-stat and p-value
np.random.seed(0)
sampleA = np.random.normal(size=100)
sampleB = np.random.normal(loc=0.1, scale=2, size=150)

# test of H0: mean = 0
result1 = ttest_1samp(sampleB, popmean=0)

# test of H0: meanA = meanB assuming equal variances
result2 = ttest_ind(sampleA, sampleB)

# test 0f H0: meanA = meanB allowing unequal variances
result3 = ttest_ind(sampleA, sampleB, equal_var=False)

In [3]:
print(result1.statistic,
result2.statistic,
result3.statistic)
print(result1.pvalue,
result2.pvalue,
result3.pvalue    
)

0.7797531026452201 -0.3089595101195194 -0.3476985381026659
0.4367730439163994 0.7576117212939927 0.7283785379267089


## Regressions with statsmodels

In [4]:
data = pd.read_csv("WAGE1_revised.csv")

In [5]:
# use result.summary() 

model1 = sm.OLS(endog=data.wage, exog=sm.add_constant(data.exper))
result1 = model1.fit()

model2 = smf.ols("wage ~ female", data=data)
result2 = model2.fit()

# multivariate
model3 = smf.ols("wage ~ female + educ", data=data)
result3 = model3.fit()

# transformations of variables
model4 = smf.ols("wage ~ female + educ + np.log(exper)", data=data)
result4 = model4.fit()

In [6]:
print(result1.summary(),
result2.summary(),
result3.summary(),
result4.summary())

                            OLS Regression Results                            
Dep. Variable:                   wage   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     6.766
Date:                Wed, 20 Sep 2023   Prob (F-statistic):            0.00955
Time:                        11:26:05   Log-Likelihood:                -1429.7
No. Observations:                 526   AIC:                             2863.
Df Residuals:                     524   BIC:                             2872.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.3733      0.257     20.908      0.0

In [7]:
# interactions
model5 = smf.ols("wage ~ female + educ + female*educ + np.log(exper)", data=data)
result5 = model5.fit()

# dummy variables
model6 = smf.ols(
    "wage ~ female + educ + female*educ + np.log(exper) + C(area)", 
    data=data
)
result6 = model6.fit()

# regression without an intercept
model7 = smf.ols(
    "wage ~ female + educ + female*educ + np.log(exper) + C(area) - 1", 
    data=data
)
result7 = model7.fit(cov_type="HC3")

In [8]:
print(result5.summary(),
result6.summary(),
    result7.summary()
)

                            OLS Regression Results                            
Dep. Variable:                   wage   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.336
Method:                 Least Squares   F-statistic:                     67.37
Date:                Wed, 20 Sep 2023   Prob (F-statistic):           6.15e-46
Time:                        11:26:05   Log-Likelihood:                -1323.4
No. Observations:                 526   AIC:                             2657.
Df Residuals:                     521   BIC:                             2678.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -3.4584      0.919     -3.765

## Regressions with linearmodels

The linearmodels package was created by Kevin Sheppard.  Many of the following examples come from the linearmodels user guide https://bashtage.github.io/linearmodels/.

### Two Stage Least Squares

In [9]:
# use result.summary 

from linearmodels.datasets import wage

data = wage.load()
data = data.dropna(subset=["educ", "wage", "sibs", "exper"])

model1 = IV2SLS.from_formula("np.log(wage) ~ exper + [educ ~ sibs]", data=data)
result1 = model1.fit(cov_type="robust")


### Seemingly Unrelated Regressions

In [10]:
from linearmodels.datasets import fringe
data = fringe.load()

formula = """ 
    {hrbens ~ educ + exper + union + south + nrtheast + nrthcen + male}
    {hrearn ~ educ + exper + nrtheast + married + male}
    """
model2 = SUR.from_formula(formula, data=data)
result2 = model2.fit(cov_type="robust")

### Panel Regressions with Fixed Effects and Clustered Standard Errors

To use the formula version of linearmodels with fixed effects, create a multi-index for the dataframe with the outside (first) index being "entity" and the inside (second) index being "time."  For other fixed effects, use the basic (non-formula) version of linearmodels.

In [11]:
from linearmodels.datasets import wage_panel
data = wage_panel.load()
data = data.set_index(["nr", "year"])

In [12]:
# entity fixed effects
model3 = PanelOLS.from_formula(
    "lwage ~ exper + EntityEffects",
    data=data,
)
result3 = model3.fit(cov_type="clustered", cluster_entity=True)

# time fixed effects
model4 = PanelOLS.from_formula(
    "lwage ~ exper + married + black + TimeEffects",
    data=data,
)
result4 = model4.fit(cov_type="clustered", cluster_time=True)

# time and entity fixed effects
model5 = PanelOLS.from_formula(
    "lwage ~ exper + EntityEffects + TimeEffects",
    data=data,
)
result5 = model5.fit(cov_type="clustered", cluster_entity=True, cluster_time=True)

### Fama-MacBeth

Run cross-sectional regressions and then use $t$ tests for the means of the time series of cross-sectional coefficients.

In [13]:
# A small data set with acc=accruals and agr=asset growth, 
# monthly data since 2010, roughly 2,000 stocks per month.

data = pd.read_csv("https://www.dropbox.com/s/012c6y4gxsxss6y/ghz.csv?dl=1", parse_dates=["date"])
data = data.sort_values(by=['permno', 'date'])
data = data.set_index(["permno", "date"])

In [14]:
# winsorize and standardize cross sections

data.agr = np.log(1+data.agr)

def winsorize(ser):
    return ser.clip(lower=ser.quantile(0.01), upper=ser.quantile(0.99))

for char in ["acc", "agr"]:
    data[char] = data.groupby("date",group_keys=False)[char].apply(winsorize)

    data[char] = data.groupby("date",group_keys=False)[char].apply(lambda x: x / x.std())



In [15]:
# run Fama-MacBeth

model6 = FamaMacBeth.from_formula("ret ~ acc + agr + 1", data=data)
result6 = model6.fit(cov_type="kernel", kernel="bartlett", bandwidth=12)

In [16]:
result6.summary

Dep. Variable:,ret,R-squared:,3.855e-05
Estimator:,FamaMacBeth,R-squared (Between):,-0.0175
No. Observations:,282598,R-squared (Within):,0.0001
Date:,"Wed, Sep 20 2023",R-squared (Overall):,3.855e-05
Time:,11:26:08,Log-likelihood,1.937e+05
Cov. Estimator:,Fama-MacBeth Kernel Cov,,
,,F-statistic:,5.4468
Entities:,3669,P-value,0.0043
Avg Obs:,77.023,Distribution:,"F(2,282595)"
Min Obs:,1.0000,,
Max Obs:,144.00,F-statistic (robust):,0.8459


In [17]:
# Fama-MacBeth "by hand" 

# cross-sectional regression function
def xreg(df):
    model = smf.ols("ret ~ acc + agr", data=df)
    result = model.fit()
    return result.params

# apply to each cross-section to get time series of coefficients
fm = data.groupby('date').apply(xreg)

# run t-tests by OLS to get Newey-West standard errors
model7a = smf.ols("acc ~ 1", data=fm)
result7a = model7a.fit(cov_type='HAC', cov_kwds={"kernel": "bartlett", "maxlags": 12})

model7b = smf.ols("agr ~ 1", data=fm)
result7b = model7b.fit(cov_type='HAC', cov_kwds={"kernel": "bartlett", "maxlags": 12})

## Rolling Window Betas

For Fama-MacBeth, we run cross-sectional regressions at each date.  For this exercise, we will run time-series regressions for each entity (stock).  We'll run the time-series regressions over rolling windows. 

The time series regressions are for stock returns on Fama-French factors.  It is common to use 60 months as the window but to include all stock/months for which 24 past months were available in the prior 60 months.  We do that with window=60, min_nobs=24, and expanding=True.

The RollingOLS function crashes if you specify a window size that is larger than the number of rows in the data frame.  So, we construct a function to "pass" if the number of rows is less than 24 and specify the window size as the smaller of 60 and the number of rows.

In [18]:
ff = pdr.DataReader("F-F_Research_Data_Factors", "famafrench", start=2000)[0] / 100
ff.index = ff.index.to_timestamp()
data = data.merge(ff, left_on='date', right_index=True) 
data = data.rename(columns={"Mkt-RF": "Mkt_RF"})
data["ret_RF"] = data.ret - data.RF
data = data.dropna(subset=["ret_RF", "Mkt_RF", "SMB", "HML"])

C:\Users\rra3\AppData\Local\Temp\ipykernel_17144\2429077064.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff = pdr.DataReader("F-F_Research_Data_Factors", "famafrench", start=2000)[0] / 100
C:\Users\rra3\AppData\Local\Temp\ipykernel_17144\2429077064.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff = pdr.DataReader("F-F_Research_Data_Factors", "famafrench", start=2000)[0] / 100


In [19]:
data

,,ret,acc,agr,Mkt_RF,SMB,HML,RF,ret_RF
permno,date,,,,,,,,
10025.0,2012-10-01,0.055125,-0.447944,0.863546,-0.0176,-0.0115,0.0359,0.0001,0.055025
10026.0,2012-10-01,-0.001047,-0.758189,0.658168,-0.0176,-0.0115,0.0359,0.0001,-0.001147
10032.0,2012-10-01,-0.111588,-0.862389,0.055487,-0.0176,-0.0115,0.0359,0.0001,-0.111688
10051.0,2012-10-01,-0.111462,-0.095727,0.311587,-0.0176,-0.0115,0.0359,0.0001,-0.111562
10104.0,2012-10-01,-0.010172,-0.609888,0.903100,-0.0176,-0.0115,0.0359,0.0001,-0.010272
...,...,...,...,...,...,...,...,...,...
93374.0,2021-12-01,0.061472,-0.382007,0.435100,0.0310,-0.0167,0.0326,0.0001,0.061372
93397.0,2021-12-01,0.031122,-0.155105,-0.107664,0.0310,-0.0167,0.0326,0.0001,0.031022
93423.0,2021-12-01,0.164342,-0.983631,-0.160819,0.0310,-0.0167,0.0326,0.0001,0.164242


In [20]:
def rolling_betas(df):
    n = df.shape[0]
    if n >= 24:
        data = df.set_index("date") 
        model = RollingOLS.from_formula(
            "ret_RF ~ Mkt_RF + SMB + HML",
            window=min(n, 60),
            min_nobs=24,
            expanding=True,
            data=data
        )
        result = model.fit()
        return result.params[['Mkt_RF', 'SMB', 'HML']].dropna()
    else:
        pass
    
betas = data.reset_index().groupby("permno").apply(rolling_betas)

In [21]:
betas

Mkt_RF       SMB       HML
permno  date                                    
10026.0 2010-01-01  0.278995 -0.115467  0.099882
        2010-02-01  0.272986 -0.114337  0.088774
        2010-03-01  0.238124 -0.104464  0.083597
        2010-04-01  0.171944  0.105564  0.107068
        2010-05-01  0.335833  0.011720  0.157090
...                      ...       ...       ...
93436.0 2021-08-01  2.173161  0.095906 -1.054570
        2021-09-01  2.081935  0.078325 -0.902165
        2021-10-01  2.185842 -0.219507 -0.842312
        2021-11-01  2.184353 -0.228243 -0.854086
        2021-12-01  2.170503 -0.141162 -0.958775

[209244 rows x 3 columns]

## Vector Auto-Regressions

In [22]:
model = VAR(ff)
result = model.fit()

In [23]:
result.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Wed, 20, Sep, 2023
Time:                     11:26:34
--------------------------------------------------------------------
No. of Equations:         4.00000    BIC:                   -36.0535
Nobs:                     282.000    HQIC:                  -36.2082
Log likelihood:           3539.40    FPE:                1.69821e-16
AIC:                     -36.3118    Det(Omega_mle):     1.58292e-16
--------------------------------------------------------------------
Results for equation Mkt-RF
               coefficient       std. error           t-stat            prob
----------------------------------------------------------------------------
const             0.010492         0.003670            2.859           0.004
L1.Mkt-RF         0.006275         0.061990            0.101           0.919
L1.SMB            0.026587         0.089927            0.296           0.76

## Saving Results

In [24]:
data = pd.read_csv("WAGE1_revised.csv")

# transformations of variables
model1 = smf.ols("wage ~ female", data=data)
result1 = model1.fit()

# multivariate
model2 = smf.ols("wage ~ female + educ + female*educ", data=data)
result2 = model2.fit()

# saving to Excel
pd.DataFrame(result2.summary().tables[1]).to_excel("excelfile.xlsx")


In [25]:
result2.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,0.2005,0.844,0.238,0.812,-1.457,1.858
female,-1.1985,1.325,-0.905,0.366,-3.802,1.405
educ,0.5395,0.064,8.400,0.000,0.413,0.666
female:educ,-0.0860,0.104,-0.830,0.407,-0.290,0.118


In [26]:
pd.__version__


'2.0.3'

In [27]:
# saving to tex

pystout(
    models=[result1, result2], 
    file="texfile.tex",
    endog_names = ["wage", "wage"],
    exogvars=[
        'female', 
        'educ', 
        'female:educ', 
        ],
    stars={0.1: "*", 0.05: "**", 0.01: "***"},
    addnotes=["$^*p<0.1$, $^{**}p<0.05$, $^{***}p<0.01$"],
    modstat={"nobs": "Obs", "rsquared_adj": "Adj $R^2$"},
    title="Wage Equation",
    label="tab:wage"

)

AttributeError: 'DataFrame' object has no attribute 'append'